## Prod AGOL Data Refresh - Hosted Flyrs.


In [ ]:
# Import libraries and connect to AGOL
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
gis = GIS("home")
import os
import logging
import time
from datetime import datetime

In [ ]:
#### PROD AGOL Hosted Flyrs ####
agol_config = {'amdw_aviation_upd': '68952a17388c476d96cab27a31c9cd38',
               'amdw_marine_upd': '69942b3093764932bfea4682f4137e93',
               'amdw_motor_vehicles_active_upd':'ea2ffa75ff9c43e6a8fe5dbd91ca3510',
               'rpdw_buildings_upd': '3a374f1845fc469ab72fe060ee5b90b3',
               'rpdw_land_upd': '01d0cf7422db44759eabdd7a23cc57ee',
               'rpdw_lease_equivalents_upd': 'c26e6a382a8a44a2bfc7bf79ad3cbdff',
               'rpdw_structures_general_upd': '519813cc901744348d36ca85ca89e619',
               'rpdw_structures_navig_aids_upd': '363afac2ee79464a936a3121c6958e58',
               'rpdw_assets_current_upd': '3e4b71cb5f60413eb3f05296478dffef',
               'gsa_cmbnd_main_brdg2_upd': '0f17bbc4981d4a1cbf6e49052e7bcf4d',
               'gsa_sa_cmbnd_brdg2_upd': '0c9a1bef126c47f7b791aef4f8ae43e2',
               'gsa_rwa_cmbnd_brdg2_upd': '1efa15f2bc484532a318a1dec58272e6',
               'gsa_lpp_cmbnd_brdg2_upd': '027596efe4084b17870891cffeb77df2',
               'gsa_lca_cmbnd_brdg2_upd': '317e7e63903a41299eeade9ac9c48005',
               'gsa_cpp_cmbnd_brdg2_upd': '34971f466eac44a8b2b3e946ee7148d9',
               'gsa_brd_cmbnd_brdg2_upd': '5e7ed6f1a4c94a07b5e26c6e9ea11a3f',
               'all_personnel_by_dstation_upd': '546c6ae9ef2f4909bd45650d5ed48663',
               'chco_fed_emp_by_dstation_upd': 'e36d9207005e4ffcaa6fbc41712d54a8',
               'chco_fed_emp_by_reszip_upd': 'ae5233563c4d426cb07a03312dffe8fb',
               'chco_federal_one_employee_upd': 'e5069788e4ad44e89b8605c81f13a582',
               'chco_federal_two_employee_upd': '9a045bebc8fa4ab7baaefdb6878e3f2d',
               'cso_military_by_dstation_upd': '9051fcec00734eb8aab32579f113320a',
               'cso_military_personnel_upd': 'd732b03be02b4898ab6092049c4da416'
              }


In [ ]:
def get_agol_item_name(agol_item_type, agol_item_name):
    try:
        search_result = gis.content.search(item_type=agol_item_type, query='title:'+agol_item_name)
        return search_result
    except Exception as e:
        logging.error('Failed to search the %s name for %s. Reason: %s' % (agol_item_type, agol_item_name, e))

In [ ]:
def set_logger():
    try:
        logger = logging.getLogger()
        log_file_name = r'/arcgis/home/agol_feature_layer_data_refresh_' + datetime.now().strftime('%Y%m%d_%H%M')+'.log'
        logformat = '%(levelname)s|%(asctime)s|%(filename)s|%(funcName)s|%(message)s'
        logging.basicConfig(filename= log_file_name, format = logformat, level=logging.INFO)
    except Exception as e:
        logging.error('Failed to set the logger. Reason: %s' % (e))
        print('Error while setting the logger')

In [ ]:
def derive_feature_layer_name(csv_name):
    try:
        return str.replace(str.replace(csv_name, '.csv', ''), '_upd', '')
    except Exception as e:
        logging.error('Failed to derive hosted feature layer name for %s. Reason: %s' % (csv_name, e))

In [ ]:
def get_feature_layer_collection(feature_layer_collection):
    try:
        return FeatureLayerCollection.fromitem(feature_layer_collection)
    except Exception as e:
        logging.error('Failed to search the hosted feature layer id for %s. Reason: %s' % (feature_layer_collection, e))

In [ ]:
def overwrite_feature_layer_collection(feature_layer_collection, csv_item_id):
    try:
        refresh_status = feature_layer_collection.manager.overwrite(csv_item_id)
        return refresh_status.get('success')
    except Exception as e:
        logging.error('Error while refreshing the data in hosted feature layer for %s with %s. with return status: %s. Reason: %s' % (feature_layer_collection, csv_item_id, refresh_status.get('success'), e))

In [ ]:
def get_item_name(item_id):
    try:
        return gis.content.get(item_id)
    except Exception as e:
        logging.error('Failed to search the hosted feature layer id for %s. Reason: %s' % (item_id, e))

In [ ]:
def delete_csv_item(csv_item_id):
    try:
        item_for_deletion = gis.content.get(csv_item_id)
    except Exception as e:
        logging.error('Failed to derive csv item id csv item for %s. Reason: %s' % (csv_item_id, e))
    try:
        delete_status = item_for_deletion.delete()
        return delete_status
    except Exception as e:
        logging.error('Failed to delete the csv item for %s. Reason: %s' % (csv_item_id, e))      

In [ ]:
data_refresh_csv_files = get_agol_item_name('CSV', 'upd')
number_of_data_files_to_process = len(data_refresh_csv_files)

In [ ]:
if len(data_refresh_csv_files)==0:
    #logging.info('No data files to process. sleeping...') 
    print('No data files to process. sleeping...')
else:
    set_logger()
    logging.info('Number of data files to be processed: %s' % (number_of_data_files_to_process)) 
    print('Number of data files to be processed: %s' % (number_of_data_files_to_process)) 
    file_number = 1
    for data_refresh_csv_file in data_refresh_csv_files:
        print('------:: Processing the csv file item #:: ' + str(file_number) + ' of ' + str(len(data_refresh_csv_files)) + ' :: ' + data_refresh_csv_file.title + ' ::------') 
        file_number += 1
        #getting item title from the contents for the CSV file

        csv_data_file_name = data_refresh_csv_file.title
        logging.info('csv_data_file_name: %s' % (csv_data_file_name))
        print('    csv_data_file_name:', csv_data_file_name)

        try:
            feature_layer_id = agol_config[csv_data_file_name]
            logging.info('feature_layer_id: %s' % (feature_layer_id))
            print('    feature_layer_id:', feature_layer_id)
        except Exception as e:
            logging.error('Could not find the item id for %s - Please update the dictionary agol_config. Reason: %s' % (csv_data_file_name, e))
            print('Could not find the item id for %s - Please update the dictionary agol_config. Reason: %s' % (csv_data_file_name, e))
            
        #get the name of the hosted feature layer to be updated
        feature_layer_name =get_item_name(feature_layer_id)
        logging.info('feature_layer_name: %s' % (feature_layer_name))
        print('    feature_layer_name:', feature_layer_name)

        #get the name of the hosted feature layer collection to be updated
        feature_layer_collection = get_feature_layer_collection(feature_layer_name)
        feature_layer_collection.layers
        logging.info('feature_layer_collection: %s' % (feature_layer_collection))
        print('    feature_layer_collection:', feature_layer_collection)
        
        ##### Testing Back up of the symbology ######################################
        item_data = feature_layer_name.get_data()
        #print('item_data', item_data)
        if item_data is not None:
            # Here note we are changing a specific part of the Layer Definition
            #layer_def = item_data['layers'][0]['layerDefinition']
            try:
                print("    ******************* SYMBOLOGY OPERATIONS *********************")
                drawingInfo_out = str(feature_layer_name.layers[0].properties.drawingInfo)
            except Exception as e:
                logging.error('Could not find the drawingInfo_out info. Reason: %s' % (e))
                print('Could not find the drawingInfo_out info. Reason: %s' % (e))
            
            #print(drawingInfo_out)
        ####### End symbology backup ##########################################


        #get the name of the csv file to be used for the update
        csv_item_name = get_agol_item_name(agol_item_type="CSV", agol_item_name=csv_data_file_name)
        logging.info('csv_item_name: %s' % (csv_item_name))
        print('    csv_item_name:', csv_item_name)

        #get the item id of the csv file to be used for the update
        csv_item_id = csv_item_name[0].id
        csv_item_name_for_update = get_item_name(csv_item_id)
        logging.info('csv_item_name_for_update: %s' % (csv_item_name_for_update))
        print('    csv_item_name_for_update:', csv_item_name_for_update)

        #copy the csv file to temp folder for the updates
        tempcsvFilePath = csv_item_name_for_update.download('/arcgis/home/temp')
        logging.info('tempcsvFilePath: %s' % (tempcsvFilePath))
        print('    tempcsvFilePath:', tempcsvFilePath)
        
        # Feature layer title capture prior to overwrite
        flyr_title = str(feature_layer_name.title)
        print('    Feature Layer Title: ' + flyr_title)
        
        # Get thumbnail #######
        flyr_thumbnail = feature_layer_name.download_thumbnail(save_folder=None)

        # overwrite the original FS with our updated CSV
        refresh_status = overwrite_feature_layer_collection(feature_layer_collection, tempcsvFilePath)
        if refresh_status:
            logging.info('data refresh for the hosted feature layer %s (Id: %s) is complete' % (str(feature_layer_name.title), str(feature_layer_id.title)))
            print('    data refresh for the hosted feature layer: ' + str(feature_layer_name.title) + '(Id: ' + str(feature_layer_id.title) + ') is complete')
            feature_layer_collection.content_status = 'authoritative'
            update_capabilities = {'capabilities': 'Query, Extract'} 
            feature_layer_collection.manager.update_definition(update_capabilities)
            logging.info('Setting capabilities for query and extract is complete')
            print('    Setting capabilities for query and extract is complete')
            ############### Adding drawing/symbology back to flyr ####################
            feature_layer_collection.layers[0].manager.update_definition({"drawingInfo": drawingInfo_out})
            logging.info('Adding Symbology back to feature layer is complete')
            print('    Adding Symbology back to feature layer is complete')
            ############### Add Title back to flyr ####################
            feature_layer_name.update(item_properties = {'title': flyr_title})
            logging.info('Adding Title back to feature layer is complete')
            print('    Adding Title back to feature layer is complete')
            ############### Add Thumbnail back to flyr ####################
            feature_layer_name.update(thumbnail = flyr_thumbnail)
            logging.info('Adding Thumbnail back to feature layer is complete')
            print('    Adding Thumbnail back to feature layer is complete')
            #csv_item_name_for_update.move('agol_archive_data_files')
            delete_csv_file = delete_csv_item(csv_item_id)
            logging.info('Moved the csv item %s to agol_archive_data_files' % (csv_item_id))
logging.info('Process completed')
print('Process completed')
